In [197]:
import os
import json
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import requests
import re
import nltk
import numpy as np

In [198]:
# Function to reset index when we split the dataframes
def drop_index(df):
    df.reset_index(drop=True, inplace=True)
    return df

In [199]:
# Read in the article dataframe and contents scraped by Newspaper
df = pd.read_csv('web_scrape/contents.csv')

In [200]:
# Clean html column for those pesky \n values
df['html'] = df['html'].replace('\n', ' ', regex=True).replace('Advertisement ', '', regex=True)

In [201]:
# Regex to remove any photo text that is at the beginning of the article
# Unfortunately, cannot grab all of them so some will eventually get in...
# Will try to make Photo a stopword
for i in range(len(df['html'])):
    r = re.findall(r'(?<=\( Shutterstock \))(.*)', df['html'][i])
    b = re.findall(r'(?<=\( Getty Images \))(.*)', df['html'][i])
    if r:
        df['html'][i] = r[0]
    elif b:
        df['html'][i] = b[0]
    else:
        continue

In [202]:
# Uncomment line below to see the sources in the notebook
#df['sources'].unique()

# Separating articles with no sources attached 
no_source = df[df['source'].isna()]
no_source = drop_index(no_source)

In [203]:
# Separating our sources into indiviudal dataframes 
wired = drop_index(df[df['source'] == 'wired'])
the_verge = drop_index(df[df['source'] == 'the_verge'])
bbc_news = drop_index(df[df['source'] == 'bbc-news'])
cnn = drop_index(df[df['source'] == 'cnn'])
techcrunch = drop_index(df[df['source'] == 'techcrunch'])
mashable = drop_index(df[df['source'] == 'mashable'])
engadget = drop_index(df[df['source'] == 'engadget'])

In [204]:
# For those that have no sources, we are going to use regex to grab them from the URL
for i in range(len(no_source['url'])):
    r = re.findall(r'(.*)(?<=.com)', no_source['url'][i])
    b = re.findall(r'(.*)(?<=.com)', no_source['url'][i]) 
    source = re.findall(r'(?<=\.)(.*)(?<=\.)', r[0])
    source = source[0].replace(r'.', '')
    if source == '':
        source = re.findall(r'(?<=\/\/)(.*)(?<=\.)', b[0])
        source = source[0].replace(r'.', '')
        no_source['source'][i] = source
    else:
        no_source['source'][i] = source
    

C:\Users\rachehuang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [207]:
# Uncomment this line to see the additional sources that we missed
no_source.source.unique()

array(['lifehacker', 'nytimes', 'gizmodo'], dtype=object)

In [208]:
# Now, we will put those into their own new dataframes. So far, we have 10 sources
lifehacker = drop_index(no_source[no_source['source'] == 'lifehacker'])
nytimes = drop_index(no_source[no_source['source'] == 'nytimes'])
gizmodo = drop_index(no_source[no_source['source'] == 'gizmodo'])

In [210]:
# Checking to make sure there are no new sources when we run the API again
df.source.unique()

array([nan, 'wired', 'the-verge', 'bbc-news', 'cnn', 'techcrunch',
       'mashable', 'engadget'], dtype=object)

In [220]:
lifehacker.key_words[7]

"['theres', 'youre', 'covid', 'coronavirus', 'rules', 'risk', 'precautions', 'perfect', 'think', 'bubble', 'right', 'cases']"